In [2]:
import os
import open3d as o3d
import numpy as np
import pickle
import pymeshlab
from shutil import copyfile

In [3]:
ycb_dir = "ycb"

# Path to the directory with all the folders with the pickle pressure files
pressure_path = "C:/Users/lucas/Desktop/UPC/MIT/manopth/outputs/graspit_to_mano/ycb/"

# Convert pressure info to pointcloud

In [5]:
pressure_points_path = os.path.join(ycb_dir, "6_pressure_pointcloud")

if not os.path.exists(pressure_points_path):
    os.makedirs(pressure_points_path)

In [6]:
get_obj_name = lambda name : name[:len(name) - 14]
obj_names = []
grasps_dict = {}

for obj_folder in os.listdir(pressure_path):   
    pkls_path = os.path.join(pressure_path, obj_folder)
    obj_name = get_obj_name(obj_folder)
    obj_names.append(obj_name)
    
    # Get sensor info from every pkl file
    all_sensors_xyz = []
    all_sensors_pressure = []
    start = 0
    end = 0

    grasps = os.listdir(pkls_path)
    n_grasps = len(grasps)
    obj_grasps_dict = {}
    for pkl_name in grasps:
        pkl_num = int(pkl_name.split("_")[-1][:-4])
        pkl_path = os.path.join(pkls_path, pkl_name)

        pressure_info = pickle.load( open( pkl_path, "rb") )

        # Load the sensors pressure and take only the ones that have pressure greater than 0
        sensors_pressure = pressure_info['sensors_pressure']
        idx_sensors = np.nonzero(sensors_pressure)
        
        # Save sensor location and pressure
        sensors_xyz = pressure_info['sensors_xyz'][idx_sensors]
        sensors_pressure = sensors_pressure[idx_sensors]
        
        # Append to list of all locations and pressure
        all_sensors_xyz.append(sensors_xyz)
        all_sensors_pressure.append(sensors_pressure)
        
        # Save how many points correspond to each grasp
        end = end + sensors_xyz.shape[0]
        obj_grasps_dict[pkl_num] = start, end
        start = end
    
    grasps_dict[obj_folder[:-14]] = obj_grasps_dict
    
    # Generate pointcloud.npz files for Convolutional Occupancy Networks 
    pc_pth = os.path.join(ycb_dir, "4_pointcloud", f"{obj_name}.npz")
    pc_npz = np.load(pc_pth)
    
    # Load scale and translation to apply to the input pointcloud
    translation = pc_npz['loc'].tolist()
    scale = pc_npz['scale'].item()
    
    sensors_xyz = np.concatenate(all_sensors_xyz)
    sensors_pressure = np.concatenate(all_sensors_pressure)

    # Point cloud of the geneated object
    pcd_gen = o3d.geometry.PointCloud()
    pcd_gen.points = o3d.utility.Vector3dVector(sensors_xyz)

    pcd_gen.estimate_normals()
    pcd_gen.orient_normals_consistent_tangent_plane(50)
    
    points = (np.asarray(pcd_gen.points) - translation) / scale
    normals = np.asarray(pcd_gen.normals)
        
    npz_pth = os.path.join(pressure_points_path, obj_name)
    np.savez(npz_pth, 
             points=points, 
             normals=normals,
             loc=pc_npz['loc'],
             scale=pc_npz['scale']
             )

# Create dataset

In [7]:
dataset_pth = os.path.join(ycb_dir, "ycb_con")

if not os.path.exists(dataset_pth):
    os.makedirs(dataset_pth)

In [8]:
for obj_name in obj_names:
    obj_dir = os.path.join(dataset_pth, obj_name)
    
    if not os.path.exists(obj_dir):
        os.makedirs(obj_dir)
        
    src_pth_ps = os.path.join(ycb_dir, "4_points", f"{obj_name}.npz")
    dst_pth_ps = os.path.join(obj_dir, "points.npz")
    copyfile(src_pth_ps, dst_pth_ps)
    
    src_pth_pcd = os.path.join(pressure_points_path, f"{obj_name}.npz")
    dst_pth_pcd = os.path.join(obj_dir, "pointcloud.npz")
    copyfile(src_pth_pcd, dst_pth_pcd)

# Save json info

In [15]:
import json

# First key is the object name
# Second key is the grasp number and how many points correspond to that grasp
json_pth = os.path.join(ycb_dir, "grasps_list.json")
with open(json_pth, 'w') as fp:
    json.dump(grasps_dict, fp, sort_keys=True)

In [18]:
import pickle

pkl_pth = os.path.join(ycb_dir, "grasps_list.p")
pickle.dump( grasps_dict, open( pkl_pth, "wb" ) )